In [34]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from Keys_and_APIs import *
from lat_lon_parser import parse
from datetime import datetime, timedelta
from dateutil import parser
from helpful_functions import *

# &nbsp; Project: Gans Web_data_Google_Cloud

# 2.4.&nbsp; Project: Location / API

In [36]:
list_of_cities_t = [('Amsterdam', 'NL'),('Bangkok', 'TH'),('Berlin', 'DE'),
                    ('Paris', 'FR'),('Sao Paulo', 'BR')]

In [5]:
#list_of_cities_t = [('Berlin', 'DE')]

In [59]:
#Final API code to the cities

# URl for city calls
city_url = "https://api.api-ninjas.com/v1/city"
city_header = {"X-Api-Key": X_Api_Key_2}

#List to construct
city_data = []

#Unpacking the tupple
for city_name, country in list_of_cities_t:
    #API Parameters
    querystring = {'name': city_name,
                   'country': country}

    #Request
    city_response = requests.request("GET",
                                    city_url,
                                    headers=city_header,
                                    params=querystring)
    #convert to JSON
    city_json = city_response.json()[0]

    #Transform the info into dictionary
    new_city = {'city_name':city_name,
                'country':city_json['country'],
                'latitude':city_json['latitude'],
                'longitude':city_json['longitude'],
               }
    #Creating Dictionary
    city_data.append(new_city)

#Final DataFrame
city_location_df = pd.DataFrame(city_data)

#Sending to SQL GC
schema = "sql_project_Wdata_gcp"
host = "34.140.142.128"
user = "root"
password = password_SQL_gcp
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

update_table('location',
             city_location_df,
             ['city_name', 'country'],
             connection_string)

city_location_df_from_sql = pd.read_sql("location", con=connection_string)
city_location_df

,city_name,country,latitude,longitude
0,Amsterdam,NL,52.3500,4.9166
1,Bangkok,TH,13.7500,100.5170
2,Berlin,DE,52.5167,13.3833
3,Paris,FR,48.8566,2.3522
4,Sao Paulo,BR,-23.5504,-46.6339


# 3.3.&nbsp; Project: Population / API

In [60]:
city_url = "https://api.api-ninjas.com/v1/city"
city_header = {"X-Api-Key": X_Api_Key_2}

#Defnining Time
today = datetime.now().date()

#List to construct
population_data = []

#Unpacking the tupple
for city_name, country in list_of_cities_t:
    #API Parameters
    querystring = {'name': city_name,
                   'country': country}

    #Request
    city_response = requests.request("GET",
                                    city_url,
                                    headers=city_header,
                                    params=querystring)
    #convert to JSON
    city_json = city_response.json()[0]

    #Transform the info into dictionary
    pop_city = {'city_name':city_name,
                'country':city_json['country'],
                'population':city_json['population'],
                'population_timestamp':today,
               }
    #Creating Dictionary
    population_data.append(pop_city)

#Final DataFrame
population_df_1 = pd.DataFrame(population_data)

# Transforming population_timestamp into date type.
population_df_1['population_timestamp'] = pd.to_datetime(population_df_1['population_timestamp'])
# Merging population with city_location's table.
population_df = city_location_df_from_sql.merge(population_df_1, on='city_name', how='right')
# Dropping columns
population_df.drop(['latitude', 'longitude', 'country_y'], axis=1, inplace=True)
#Renaming column country_x.
population_df.rename(columns={'country_x': 'country'}, inplace=True)

#Sending to SQL GC
schema = "sql_project_Wdata_gcp"
host = "34.140.142.128"
user = "root"
password = password_SQL_gcp
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

update_table('population',
             population_df,
             ['city_name', 'country','population'],
             connection_string)


population_df

,city_id,city_name,country,population,population_timestamp
0,1,Amsterdam,NL,1031000,2024-09-16
1,2,Bangkok,TH,17066000,2024-09-16
2,3,Berlin,DE,3644826,2024-09-16
3,4,Paris,FR,11020000,2024-09-16
4,5,Sao Paulo,BR,22046000,2024-09-16


# 4.3 &nbsp; Weather Function

In [67]:
def weather_sql_cities():

    #Acessing to SQL GC
    schema = "sql_project_Wdata_gcp"
    host = "34.140.142.128"
    user = "root"
    password = password_SQL_gcp
    port = 3306
        
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    

    city_location_df_from_sql = pd.read_sql("location", con=connection_string)
    
    # Access weather site
    weather_url = "https://api.openweathermap.org/data/2.5/forecast?"
    
    weather_items = []
    
    # Loop over cities from the SQL database
    for i, row in city_location_df_from_sql.iterrows():
        city_id = row['city_id']
        lat = row['latitude']
        lon = row['longitude']
    
        querystring = {
            'lat': lat,
            'lon': lon,
            'appid': weather_API_key,
            'units': 'metric'
        }
    
        # Request weather data
        weather_response = requests.request('GET', 
                                            weather_url, 
                                            params=querystring)

        for item in weather_response.json()['list']:
            
    
            # Process each weather item in the response
                weather_item = {
                    'city_id': city_id,
                    'forecast_time': item.get('dt_txt'),
                    'temperature': item['main'].get('temp'),
                    'temperature_feels_like': item['main'].get('feels_like'),
                    'temperature_min': item['main'].get('temp_min'),
                    'temperature_max': item['main'].get('temp_max'),
                    'forecast': item['weather'][0].get('main'),
                    'precipitation_prob': item.get('pop'),
                    'rain_in_last_3h': item.get('rain', {}).get('3h', 0),
                    'snow_in_last_3h': item.get('snow', {}).get('3h', 0),
                    'wind_speed': item['wind'].get('speed'),
                }
                weather_items.append(weather_item)
    
    # Create the DataFrame from the weather items list outside the loop
    city_weather_df = pd.DataFrame(weather_items)

    #Sending to SQL    
    only_add_new('weather',
                 city_weather_df,
                 ['city_id', 'forecast_time'],
                 connection_string)

weather_sql_cities()

# 5.3 &nbsp; Project: Airports function

In [62]:
def airport_sql_cities():

    #Acessing to SQL GC
    schema = "sql_project_Wdata_gcp"
    host = "34.140.142.128"
    user = "root"
    password = password_SQL_gcp
    port = 3306
        
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    
    city_location_df_from_sql = pd.read_sql("location", con=connection_string)
    
    # Access airport site
    airport_url = f"https://aerodatabox.p.rapidapi.com/airports/search/location"

    #Creating lists
    airport_items = []
    city_airport_items = []
    
    # Loop over cities from the SQL database
    for i, row in city_location_df_from_sql.iterrows():
        city_id = row['city_id']
        lat = row['latitude']
        lon = row['longitude']
    
        querystring = {
            'lat': lat,
            'lon': lon,
            'radiusKm': '30',
            'limit': '10',
            'withFlightInfoOnly':'false'
        }
        headers = {
            'x-rapidapi-key': flights_API_Key,
            'x-rapidapi-host': "aerodatabox.p.rapidapi.com"
        }

        #Requesting the data
        airport_response = requests.request('GET', 
                                            airport_url, 
                                            headers=headers, 
                                            params=querystring)

        for item in airport_response.json()['items']:

                airport_item= {
                        'iata': item['iata'],
                        'airport_name': item.get('name'),
                    }
                
                # Append to the airports table
                airport_items.append(airport_item)

                #Bridge Table
                city_airport_item = {
                        'city_id': city_id, 
                        'iata': item['iata'],
                    }
                
                # Append to the city_airports table
                city_airport_items.append(city_airport_item)
        
        
    # Create the DataFrame from the airport items list
    airports_df = pd.DataFrame(airport_items)
    airports_df.drop_duplicates(inplace = True)
    
    city_airports_df = pd.DataFrame(city_airport_items)

    #Sending data to SQL
    only_add_new('airports',
                 airports_df,
                 ['iata', 'airport_name'],
                 connection_string)

    only_add_new('city_airports',
                 city_airports_df,
                 ['city_id', 'iata'],
                 connection_string)
    
airport_sql_cities()

# 6.3 &nbsp; Project: Flights Function

In [19]:
def flights_sql_cities():

    
    #Acessing to SQL GC
    schema = "sql_project_Wdata_gcp"
    host = "34.140.142.128"
    user = "root"
    password = password_SQL_gcp
    port = 3306
        
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    
    city_airports_df_from_sql = pd.read_sql("city_airports", con=connection_string)

    #Creating list
    flight_items = []
    
    # Calculate 'tomorrow' date
    tomorrow = datetime.now().date() + timedelta(days=1)
    
    # Time ranges
    time_ranges = [('00:00', '11:59'), ('12:00', '23:59')]
    
    # Loop over cities from the SQL database
    for i, row in city_airports_df_from_sql.iterrows():
        iata = row['iata']
        
        for start_time, end_time in time_ranges:
            date_start = f"{tomorrow}T{start_time}"
            date_end = f"{tomorrow}T{end_time}"
            
            # Access airport site
            flight_url = f'https://aerodatabox.p.rapidapi.com/flights/airports/iata/{iata}/{date_start}/{date_end}'
            
            querystring_f = {
                'withLeg':'true',
                'direction':'Both',
                'withCancelled':'true',
                'withCodeshared':'true',
                'withCargo':'true',
                'withPrivate':'true',
                'withLocation':'false'
            }
            headers = {
                'x-rapidapi-key': flights_API_Key,
                'x-rapidapi-host': "aerodatabox.p.rapidapi.com"
            }
    
            flight_response = requests.get(flight_url, headers=headers, params=querystring_f)
            if flight_response.status_code == 200:
                response_f = flight_response.json()
    
                # Process each airport item in the response
                for item in response_f.get('arrivals', []):
                    arrival_local_time = item['arrival'].get('scheduledTime').get('local')

                    #retrival time of the request
                    retrieval_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    
                    # Parse and format datetime to remove timezone information
                    parsed_time = parser.parse(arrival_local_time)
                    formatted_time = parsed_time.strftime('%Y-%m-%d %H:%M:%S')  # Adjust format as needed
    
                    flight_item = {
                        'local_airport': iata,
                        'origin_airport': item['departure'].get('airport').get('iata'),
                        'airport_name': item['departure'].get('airport').get('name'),                        
                        'arrival_local_time': formatted_time,
                        'airline': item['airline'].get('name'),
                        'flight_number': item['number'],
                        'data_retrieved_at': retrieval_time
                    }
                    # Append the airport item to the list
                    flight_items.append(flight_item)
        
    # Create the DataFrame from the airport items list
    city_flights_df = pd.DataFrame(flight_items)

    
    # Sending data to SQL
    only_add_new('flights',
                 city_flights_df,
                 ['local_airport', 'arrival_local_time','flight_number'],
                 connection_string)

flights_sql_cities()